In [23]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
from music21 import instrument, note, stream, chord, converter,midi, duration,volume
import copy
import random

import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, Embedding


In [24]:
data_directory = "DataX/"
charIndex_json = "char_to_index.json"
model_weights_directory = 'DataX/Model_Weights/'
BATCH_SIZE = 16


In [25]:
def make_model(unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(128, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add((Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [26]:
# main function for pg2
def generate_sequence(epoch_num, initial_index, seq_length):
    padder = int(seq_length * 0.2)
    seq_length += padder
    with open(os.path.join(data_directory, charIndex_json)) as f:
        char_to_index = json.load(f)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    unique_chars = len(index_to_char)
    
    model = make_model(unique_chars)
    model.load_weights(model_weights_directory + "Weights_{}.h5".format(epoch_num))
     
    sequence_index = [initial_index]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)

        
        sequence_index.append(sample[0])
    
    output_sequence = [index_to_char[c] for c in sequence_index]
    output_sequence = output_sequence[padder:]
    return output_sequence
    

In [27]:
def humanize(seq_len):
    # randomize duration and velocity
    mu, sigma = 1, 0.15 # mean and standard deviation
    duration_list = np.random.normal(mu, sigma, seq_len)
    mu, sigma = 75, 9 
    velocity_list = np.random.normal(mu, sigma, seq_len)
    velocity_list = [int(i) for i in velocity_list]
    return duration_list, velocity_list


In [28]:
def humanize_chord(vel,seq_len):
    # randomize the velocity for each value in chord
    mu, sigma = vel, 3 # mean and standard deviation
    velocity_list = np.random.normal(mu, sigma, seq_len)
    velocity_list = [int(i) for i in velocity_list]
    return velocity_list

In [29]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []
    
    # get a list of the indexes of all chords
    l_chords = []
    for i in range(len(prediction_output)):
        if ('.' in prediction_output[i]) or prediction_output[i].isdigit():
            l_chords.append(i)       
    last_chord = None
    
    duration_list, velocity_list = humanize(len(prediction_output))
    
    
    # create note and chord objects based on the values generated by the model
    for i in range(len(prediction_output)):
        
        # pattern is a chord
        if ('.' in prediction_output[i]) or prediction_output[i].isdigit():
            notes_in_chord = prediction_output[i].split('.')
            notes_in_chord = [int(i) for i in notes_in_chord]
            soprano = max(notes_in_chord)
            notes = []   
            chord_vel = humanize_chord(velocity_list[i],len(notes_in_chord))
            if i != l_chords[-1]:
                # when the current chord is not the last chord
                curr_idx = l_chords.index(i)
                time = (l_chords[curr_idx+1] - i) * 0.5
                if time > 2:
                    time = 2
                
                for current_note in notes_in_chord:
                    if current_note != soprano:
                        new_note = note.Note(current_note,quarterLength=time)
                    else:
                        new_note = note.Note(current_note)
                    new_note.volume = volume.Volume(velocity=random.choice(chord_vel))
                    new_note.storedInstrument = instrument.Piano()   # change the instrument
                    notes.append(new_note)
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
                
            else:
                last_chord = copy.deepcopy(notes_in_chord)
                continue
                          
        # pattern is a note
        else:
            new_note = note.Note(prediction_output[i],quarterLength=duration_list[i])
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
#             new_note.storedInstrument = instrument.Vocalist()
            new_note.volume = volume.Volume(velocity=velocity_list[i])

            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5
        
    
    if last_chord is not None:
        for current_note in last_chord:
            new_note = note.Note(current_note)
#             new_note.storedInstrument = instrument.Piano()   
            notes.append(new_note)
        new_chord = chord.Chord(notes,quarterLength=1)
        new_chord.offset = offset
        output_notes.append(new_chord)
        

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')



In [30]:
def generate():
    # generate function for pg1
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes_given', 'rb') as filepath:
        notes = pickle.load(filepath)

    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))


    # get output sequence
    initial_index = np.random.randint(low=1, high=n_vocab-1, size=1)
    # epoch_num: identify the model to be loaded
    # initial_index: randomize starting index
    # seq_length: length of output
    output_sequence = generate_sequence(epoch_num=320, initial_index=initial_index[0], seq_length=400)
    return output_sequence
    

In [31]:
output_sequence = generate()

In [32]:
create_midi(output_sequence)

In [33]:
mf = midi.MidiFile()
mf.open('test_output.mid')
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
s.show('midi')
# this midi player shows little variation in velocity

# References
1. [Data source] "Giovanni Pierluigi da Palestrina" *ChoralWiki.* 
    <br>http://www1.cpdl.org/wiki/index.php/Giovanni_Pierluigi_da_Palestrina <br>
2. [LSTM model] Gaurav Sharma. "Music Generation Using Deep Learning." *Medium.*
    <br>https://medium.com/datadriveninvestor/music-generation-using-deep-learning-85010fb982e2?<br>
3. [Data processing] Sigurður Skúli. "How to Generate Music using a LSTM Neural Network in Keras" *Medium.*
    <br>https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5<br>
4. [WebScraper] Samridha Shretha. *Github.*
    <br>https://github.com/SamSamhuns/musical_python/blob/master/scrap_midi/scrap_freemidi_org.py<br>